In [ ]:
import os
import cv2

def generate_fall_clips(root_dir, output_dir, num_clips_per_vid=1):
    """
    Generate fall clips from videos in the dataset based on annotation files.

    Args:
        root_dir (str): Root directory of the dataset (e.g., 'archive').
        output_dir (str): Directory where extracted clips will be saved (e.g., 'processed/fall/video').
        num_clips_per_vid (int): Number of clips to extract per video. Currently supports only 1.

    The function traverses the dataset, finds subfolders with 'Annotation_files' and 'Videos',
    and extracts video segments based on start and end frames from annotation files.
    """
    # Check if num_clips_per_vid > 1 is requested (not supported yet)
    if num_clips_per_vid != 1:
        print("Warning: num_clips_per_vid > 1 is not supported yet. Extracting one clip per video.")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Traverse the root directory to find folders with both 'Annotation_files' and 'Videos'
    for root, dirs, files in os.walk(root_dir):
        # Handle case sensitivity and common naming variations
        annotation_dir_name = 'Annotation_files' if 'Annotation_files' in dirs else 'Annotations_files' if 'Annotations_files' in dirs else None
        videos_dir_name = 'Videos' if 'Videos' in dirs else None

        if annotation_dir_name and videos_dir_name:
            annotation_folder = os.path.join(root, annotation_dir_name)
            video_folder = os.path.join(root, videos_dir_name)
            location_name = os.path.basename(root)  # e.g., 'Coffee_room_01'

            # List all .txt files in the annotation folder
            annotation_files = [f for f in os.listdir(annotation_folder) if f.endswith('.txt')]

            for annotation_file in annotation_files:
                # Extract video number from annotation filename, e.g., '1' from 'video (1).txt'
                try:
                    video_number = annotation_file.split(' (')[1].split(').txt')[0]
                except IndexError:
                    print(f"Skipping invalid annotation file: {annotation_file}")
                    continue

                # Construct corresponding video filename
                video_file_name = f"video ({video_number}).avi"
                video_path = os.path.join(video_folder, video_file_name)
                groundtruth_file = os.path.join(annotation_folder, annotation_file)

                # Check if the video file exists
                if not os.path.exists(video_path):
                    print(f"Video file not found: {video_path}")
                    continue

                # Define output filename with location and video number for uniqueness
                output_file_name = f"{location_name}_video_{video_number}.mp4"
                output_video_path = os.path.join(output_dir, output_file_name)

                # Open the video file
                cap = cv2.VideoCapture(video_path)
                if not cap.isOpened():
                    print(f"Error opening video file: {video_path}")
                    continue

                # Get video properties
                fps = cap.get(cv2.CAP_PROP_FPS)
                frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                # Read start and end frames from the annotation file
                try:
                    with open(groundtruth_file, 'r', encoding='ISO-8859-1') as file:
                        lines = file.readlines()
                        start_frame = int(lines[0].strip())
                        end_frame = int(lines[1].strip())
                except Exception as e:
                    print(f"Error reading groundtruth file {groundtruth_file}: {e}")
                    cap.release()
                    continue

                # Define the segment to extract (30 frames starting 15 frames before the fall)
                segment_start = max(0, start_frame - 15)
                segment_end = min(total_frames, segment_start + 30)

                # Create video writer for the output clip
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

                # Extract and write the frames
                cap.set(cv2.CAP_PROP_POS_FRAMES, segment_start)
                for i in range(segment_start, segment_end):
                    ret, frame = cap.read()
                    if not ret:
                        print(f"Error reading frame {i} from {video_path}")
                        break
                    out.write(frame)

                # Release resources
                cap.release()
                out.release()
                print(f"Video segment saved to {output_video_path}")

# Example usage
if __name__ == "__main__":
    generate_fall_clips(root_dir='../archive', output_dir='../processed/fall/video', num_clips_per_vid=1)

Video segment saved to processed/fall/video/Home_01_video_7.mp4
Video segment saved to processed/fall/video/Home_01_video_17.mp4
Video segment saved to processed/fall/video/Home_01_video_21.mp4
Video segment saved to processed/fall/video/Home_01_video_20.mp4
Video segment saved to processed/fall/video/Home_01_video_16.mp4
Video segment saved to processed/fall/video/Home_01_video_6.mp4
Video segment saved to processed/fall/video/Home_01_video_27.mp4
Video segment saved to processed/fall/video/Home_01_video_1.mp4
Video segment saved to processed/fall/video/Home_01_video_11.mp4
Video segment saved to processed/fall/video/Home_01_video_10.mp4
Video segment saved to processed/fall/video/Home_01_video_26.mp4
Video segment saved to processed/fall/video/Home_01_video_30.mp4
Video segment saved to processed/fall/video/Home_01_video_25.mp4
Video segment saved to processed/fall/video/Home_01_video_29.mp4
Video segment saved to processed/fall/video/Home_01_video_13.mp4
Video segment saved to proce

[mp3float @ 0x123fe0900] Header missing
[mp3float @ 0x107ae4b70] Header missing
[mp3float @ 0x107ae4580] Header missing
[mp3float @ 0x14bf3f310] Header missing
[mp3float @ 0x107ae4c30] Header missing
[mp3float @ 0x107ae4580] Header missing
[mp3float @ 0x107ae4580] Header missing
[mp3float @ 0x14bf3a800] Header missing
[mp3float @ 0x14bf3a800] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing


Video segment saved to processed/fall/video/Coffee_room_01_video_20.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_36.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_41.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_16.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_6.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_31.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_27.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_1.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_11.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_46.mp4


[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x123fe0310] Header missing
[mp3float @ 0x123fe0310] Header missing
[mp3float @ 0x14bf3a2b0] Header missing
[mp3float @ 0x14bf3a2b0] Header missing
[mp3float @ 0x14bf3a2b0] Header missing
[mp3float @ 0x14bf3a2b0] Header missing
[mp3float @ 0x14bf3c730] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing


Video segment saved to processed/fall/video/Coffee_room_01_video_47.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_10.mp4
Error reading groundtruth file ../archive/Coffee_room_01/Coffee_room_01/Annotation_files/video (26).txt: invalid literal for int() with base 10: '1,1,72,58,132,170'
Video segment saved to processed/fall/video/Coffee_room_01_video_30.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_25.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_33.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_48.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_29.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_44.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_13.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_3.mp4


...

[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4b70] Header missing
[mp3float @ 0x107ae4b70] Header missing
[mp3float @ 0x107ae4c50] Header missing
[mp3float @ 0x107ae4c50] Header missing
[mp3float @ 0x107ae4c50] Header missing
[mp3float @ 0x107ae4c50] Header missing
[mp3float @ 0x107ae4580] Header missing


Video segment saved to processed/fall/video/Coffee_room_01_video_43.mp4
Video segment saved to processed/fall/video/Coffee_room_01_video_38.mp4
Video segment saved to processed/fall/video/Home_02_video_56.mp4
Video segment saved to processed/fall/video/Home_02_video_40.mp4
Video segment saved to processed/fall/video/Home_02_video_37.mp4
Video segment saved to processed/fall/video/Home_02_video_60.mp4
Video segment saved to processed/fall/video/Home_02_video_36.mp4
Video segment saved to processed/fall/video/Home_02_video_41.mp4
Video segment saved to processed/fall/video/Home_02_video_57.mp4
Video segment saved to processed/fall/video/Home_02_video_31.mp4
Video segment saved to processed/fall/video/Home_02_video_50.mp4
Video segment saved to processed/fall/video/Home_02_video_46.mp4
Video segment saved to processed/fall/video/Home_02_video_47.mp4
Video segment saved to processed/fall/video/Home_02_video_51.mp4
Video segment saved to processed/fall/video/Home_02_video_33.mp4
Video segme

[mp3float @ 0x14bf3c3b0] Header missing
[mp3float @ 0x107ae4c40] Header missing
[mp3float @ 0x107ae4580] Header missing
[mp3float @ 0x107ae4580] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing
[mp3float @ 0x107ae4900] Header missing


Video segment saved to processed/fall/video/Coffee_room_02_video_64.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_68.mp4
Error reading groundtruth file ../archive/Coffee_room_02/Coffee_room_02/Annotations_files/video (52).txt: invalid literal for int() with base 10: '1,1,201,75,267,173'
Video segment saved to processed/fall/video/Coffee_room_02_video_53.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_69.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_49.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_65.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_54.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_62.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_58.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_59.mp4
Video segment saved to processed/fall/video/Coffee_room_02_video_63.mp4
Video segment saved to processed/fall/vid

[mp3float @ 0x107ae4c20] Header missing
[mp3float @ 0x107ae4580] Header missing
